In [4]:
import sys
sys.path.append("../")
import pandas as pd
from get_trace_durations import get_trace_durations

In [7]:
class CaseDurationsManager:
    def __init__(self, case_table=None):
        self.case_table = case_table
        self.case_durations = None

    def add_case_durations(self, measurements):
        self.case_durations = pd.concat([self.case_durations, get_trace_durations(measurements)], ignore_index=True)

    def collect_case_durations(self, data_collector):
        self.case_durations = None
        mean_ranks = data_collector.get_mean_ranks()
        num_runs = 0
        if isinstance(mean_ranks, pd.DataFrame):
            num_runs = mean_ranks.shape[1] - 2

            for i in range(num_runs):
                data = data_collector.get_runtimes_competing_table(i)
                self.add_case_durations(data)

        print("Collected data from {} runs".format(num_runs))

    def get_alg_measurements(self):
        alg_list = [alg.split('_')[0] for alg in list(self.case_durations['case:concept:name'])]
        alg_list = list(set(alg_list))

        alg_measurements = {}
        for alg in alg_list:
            t_alg = self.case_durations[self.case_durations.apply(
                lambda x: x['case:concept:name'].split('_')[0] == alg, axis=1)]
            alg_measurements[alg] = list(t_alg['case:duration'])
        return alg_measurements

    def clear_case_durations(self):
        self.case_durations = None

### Input: set of measurements table

In [8]:
sys.path.append("../../data_integration/")
from data_collector import DataCollector

In [17]:
dc = DataCollector("sample_data/100_100_10_100_100/")
run_times_table0 = dc.get_runtimes_competing_table(0) # 3 measurements per table
run_times_table1 = dc.get_runtimes_competing_table(1)

In [14]:
run_times_table0.head()

,case:concept:name,concept:name,concept:flops,concept:operation,concept:kernel,timestamp:start,timestamp:end
0,algorithm2_02,gemm_2e+05,200000.0,tmp1 = (A B),"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml4)",1.657009e+09,1.657009e+09
1,algorithm2_02,gemm_2e+05,200000.0,tmp4 = (tmp1 C),"gemm!('N', 'N', 1.0, ml4, ml2, 0.0, ml5)",1.657009e+09,1.657009e+09
2,algorithm2_02,gemm_2e+06,2000000.0,tmp6 = (tmp4 D),"gemm!('N', 'N', 1.0, ml5, ml3, 0.0, ml6)",1.657009e+09,1.657009e+09
3,algorithm5_00,gemm_2e+05,200000.0,tmp2 = (B C),"gemm!('N', 'N', 1.0, ml1, ml2, 0.0, ml4)",1.657009e+09,1.657009e+09
4,algorithm5_00,gemm_2e+06,2000000.0,tmp4 = (A tmp2),"gemm!('N', 'N', 1.0, ml0, ml4, 0.0, ml5)",1.657009e+09,1.657009e+09


In [15]:
cm = CaseDurationsManager()

In [16]:
cm.add_case_durations(run_times_table0)

In [18]:
cm.add_case_durations(run_times_table1)

In [19]:
cm.get_alg_measurements() #6 measurements

{'algorithm5': [0.0002200603485107422,
  0.000202178955078125,
  0.0002129077911376953,
  0.00027298927307128906,
  0.000263214111328125,
  0.00028014183044433594],
 'algorithm3': [0.0001480579376220703,
  0.00014901161193847656,
  0.0001480579376220703,
  0.00021696090698242188,
  0.0002009868621826172,
  0.00019216537475585938],
 'algorithm2': [0.0001919269561767578,
  0.00014209747314453125,
  0.0001399517059326172,
  0.00039505958557128906,
  0.0002110004425048828,
  0.0001728534698486328],
 'algorithm0': [9.298324584960938e-05,
  0.00010013580322265625,
  7.581710815429688e-05,
  9.012222290039062e-05,
  0.00011610984802246094,
  0.00016498565673828125],
 'algorithm1': [8.392333984375e-05,
  8.392333984375e-05,
  8.487701416015625e-05,
  0.00015807151794433594,
  7.510185241699219e-05,
  8.0108642578125e-05],
 'algorithm4': [0.00020313262939453125,
  0.0002028942108154297,
  0.00020599365234375,
  0.00034689903259277344,
  0.00029397010803222656,
  0.00024199485778808594]}

In [20]:
cm.case_durations

,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration
0,algorithm2_02,1.657009e+09,1.657009e+09,0.000192
1,algorithm5_00,1.657009e+09,1.657009e+09,0.000220
2,algorithm5_01,1.657009e+09,1.657009e+09,0.000202
3,algorithm1_02,1.657009e+09,1.657009e+09,0.000084
4,algorithm4_00,1.657009e+09,1.657009e+09,0.000203
5,algorithm1_01,1.657009e+09,1.657009e+09,0.000084
6,algorithm4_01,1.657009e+09,1.657009e+09,0.000203
7,algorithm2_01,1.657009e+09,1.657009e+09,0.000142
8,algorithm4_02,1.657009e+09,1.657009e+09,0.000206
9,algorithm0_01,1.657009e+09,1.657009e+09,0.000093
